In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import copy
import pickle
import pandas as pd
from tqdm import tqdm
from typing import Counter
import math
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, recall_score, precision_score
import threading
from sklearn.svm import SVC


def load_data():
    data = loadmat("data/mnist_all.mat")

    # print(data.keys())

    train_data = pd.DataFrame()
    test_data = pd.DataFrame()

    for i in range(10):
        temp_df = pd.DataFrame(data["train" + str(i)])
        temp_df['label'] = i
        train_data = train_data.append(temp_df)
        temp_df = pd.DataFrame(data["test" + str(i)])
        temp_df['label'] = i
        test_data = test_data.append(temp_df)

    train_data = shuffle(train_data)
    test_data = shuffle(test_data)

    train_labels = np.array(train_data['label'])
    test_labels = np.array(test_data['label'])

    train_data = train_data.drop('label', axis=1)
    test_data = test_data.drop('label', axis=1)
    
    train_data = np.array(train_data) / 255
    test_data = np.array(test_data) / 255
    
    pca = PCA(0.95)
    pca.fit(train_data)
    train_data = pca.transform(train_data)
    test_data = pca.transform(test_data)

    return train_data, test_data, train_labels, test_labels


X_train, X_test, y_train, y_test = load_data()

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=0)

In [2]:
print(X_train.shape)

(48000, 154)


In [3]:
class SVM:
    def __init__(self, C):
        self.C = C
        self.b = 0
    
    def train(self, X_data, Y_data, valid_x, valid_y, epochs, kernel_type):
        self.x_num = X_data.shape[0]
        
        self.epochs = epochs
        self.X_data = X_data
        self.Y_data = Y_data
        
        self.valid_x  = valid_x
        self.valid_y = valid_y
        
        self.E = np.zeros(self.x_num)
        self.alpha = np.zeros(self.x_num)
        
        self.best_alpha = None
        self.best_f1 = 0
        self.best_b = None
        
        self.kernel_type = kernel_type
        
        self.kernel_matrix = np.zeros(shape=(self.x_num, self.x_num))
        
        self.valid_kernel_matrix = np.zeros(shape=(self.x_num, len(self.valid_y)))
        
        self.init_valid_kernel_matrix()
        self.init_kernel_matrix()
        self.init_E()
        
        pair_changed = 0
        for epoch in tqdm(range(self.epochs)):
            for i in range(self.x_num):
    
                if not ((self.Y_data[i] * self.E[i] < -0.001) and (self.alpha[i] <self.C) or \
                     (self.Y_data[i] * self.E[i] > 0.001) and (self.alpha[i] > 0)):
                    continue
                
                val, j = self.select_j(i)
                
                while j < self.x_num:
                    if i == j:
                        j += 1
                        continue
                    if self.Y_data[i] == self.Y_data[j]:
                        L = max(0, self.alpha[j] + self.alpha[i] - self.C)
                        H = min(self.C, self.alpha[j] + self.alpha[i])
                    else:
                        L = max(0, self.alpha[j] - self.alpha[i])
                        H = min(self.C, self.C + self.alpha[j] - self.alpha[i])

                    if L == H:
                        j += 1
                        continue
                    old_alpha_j = self.alpha[j]
                    yita = self.kernel_matrix[i][i] + self.kernel_matrix[j][j] \
                               - 2 * self.kernel_matrix[i][j]

                    if yita <= 0:
                        j += 1
                        continue

                    new_alpha_unc_j = old_alpha_j + self.Y_data[j] * (self.E[i] - self.E[j]) / yita

                    if new_alpha_unc_j >= H:
                        new_alpha_clippd_j = H
                    elif new_alpha_unc_j <= L:
                        new_alpha_clippd_j = L
                    else:
                        new_alpha_clippd_j = new_alpha_unc_j

                    if(np.abs(old_alpha_j-new_alpha_clippd_j) <= 1e-5):
                        j += 1
                        continue
                    
                    break
                
                if j >= self.x_num:
                    continue
                    
                self.alpha[j] = new_alpha_clippd_j
                
                old_alpha_i = self.alpha[i]
                self.alpha[i] = old_alpha_i + self.Y_data[i] * self.Y_data[j] * (old_alpha_j - self.alpha[j])

                old_b = self.b

                bi_new = -self.E[i] - self.Y_data[i] * self.kernel_matrix[i][i] * (self.alpha[i] - old_alpha_i) \
                             - self.Y_data[j] * self.kernel_matrix[i][j] * (self.alpha[j] - old_alpha_j) + old_b

                bj_new = -self.E[j] - self.Y_data[i] * self.kernel_matrix[i][j] * (self.alpha[i] - old_alpha_i) \
                             - self.Y_data[j] * self.kernel_matrix[j][j] * (self.alpha[j] - old_alpha_j) + old_b
                
                if self.alpha[i] > 0 and self.alpha[i] < self.C:
                    self.b = bi_new
                elif self.alpha[j] > 0 and self.alpha[j] < self.C:
                    self.b = bj_new
                else:
                    self.b = (bi_new + bj_new)/2.0

                self.update_E(i)
                self.update_E(j)

                
                pair_changed += 1
                if pair_changed % 10 == 0:
                    state = self.valid()
                    if state is True:
                        return
        

    def init_kernel_matrix(self):
        print('start initing kernel matrix...')
        num = self.x_num
        for i in tqdm(range(0, num)):
            for j in range(0, num):
                if i > j:
                    self.kernel_matrix[i][j] = self.kernel_matrix[j][i]
                    continue
                if self.kernel_type == 'rbf':
                    self.kernel_matrix[i][j] = self.rbf_kernel(self.X_data[i], self.X_data[j])
                elif self.kernel_type == 'poly':
                    self.kernel_matrix[i][j] = self.polynominal_kernel(self.X_data[i], self.X_data[j])
        print('finish initing kernel matrix...')
    
    
    def init_valid_kernel_matrix(self):
        print('start initing valid kernel matrix...')
        for i in tqdm(range(self.x_num)):
            for j in range(len(self.valid_y)):
                if self.kernel_type == 'rbf':
                    self.valid_kernel_matrix[i][j] = self.rbf_kernel(self.X_data[i], self.valid_x[j])
                elif self.kernel_type == 'poly':
                    self.valid_kernel_matrix[i][j] = self.polynominal_kernel(self.X_data[i], self.valid_x[j])
        print('finish initing valid kernel matrix...')
        

    def update_E(self, index):
        
        sv = self.get_support_vectors()
        s = 0
        for i in range(len(sv)):
            s += self.Y_data[i]*self.alpha[i]*self.kernel_matrix[i][index]
        s += self.b - self.Y_data[index]
        
       
        self.E[index] = s
        
        

    def init_E(self):
        print('start initing E...')
        for i in tqdm(range(self.x_num)):
            self.E[i] = self.g_x_with_kernel_matrix(i) - self.Y_data[i]
        print('finish initing E...')
    

    def select_j(self, i):
        max_abs = 0
        max_index = -1
        for j in range(self.x_num):
            if i == j:
                continue
                
            temp_abs = np.abs(self.E[i]-self.E[j])
            if temp_abs > max_abs:
                max_abs = temp_abs
                max_index = j
        
        return max_abs, max_index
    

    def predict(self, x, opt):
        s = 0
        if opt == 'valid':
            for i in range(self.x_num):
                s += self.alpha[i] * self.Y_data[i] * self.valid_kernel_matrix[i][x]
            s += self.b
        elif opt == 'test':
            for i in range(self.x_num):
                if self.kernel_type == 'rbf':
                    s += self.best_alpha[i] * self.Y_data[i] * self.rbf_kernel(self.X_data[i], x)
                elif self.kernel_type == 'poly':
                    s += self.best_alpha[i] * self.Y_data[i] * self.polynominal_kernel(self.X_data[i], x)
            s += self.best_b
        
        return np.sign(s)
    

    def g_x(self, x):
        s = 0
        for k in range(self.x_num):
            if self.kernel_type == 'rbf':
                s += self.alpha[k]*self.Y_data[k]*self.rbf_kernel(x, self.X_data[k])
            elif self.kernel_type == 'poly':
                s += self.alpha[k]*self.Y_data[k]*self.polynominal_kernel(x, self.X_data[k])
        return s + self.b
    
    

    def g_x_with_kernel_matrix(self, i):
        s = 0
        for k in range(self.x_num):
            s += self.alpha[k]*self.Y_data[k]*self.kernel_matrix[i][k]
        
        return s + self.b
    

    def rbf_kernel(self, x, z):
        gamma = 0.5     # gamma > 0
        return np.exp(-gamma*np.linalg.norm(x-z))
    

    def get_support_vectors(self):
        sv = []
        for i in range(self.x_num):
            if self.alpha[i] > 0 and self.alpha[i] < self.C:
                sv.append(i)
                
        return sv
    
    
    def predict_of_probability(self, x):
        s = 0
       
        for i in range(self.x_num):
            if self.kernel_type == 'rbf':
                s += self.best_alpha[i] * self.Y_data[i] * self.rbf_kernel(self.X_data[i], x)
            elif self.kernel_type == 'poly':
                s += self.best_alpha[i] * self.Y_data[i] * self.polynominal_kernel(self.X_data[i], x)
                
        s += self.best_b
        
        return s
    
    
    def polynominal_kernel(self, x, z):
        return math.pow(np.dot(x, z) + 1, 3)
    
    
    def valid(self):
        y_predict = []
        for i in range(len(self.valid_x)):
            p = self.predict(i, opt='valid')
            y_predict.append(p)
        
        valid_f1 = f1_score(self.valid_y, y_predict)
        print('valid f1: {} precision: {} recall: {}'.format(valid_f1 , precision_score(self.valid_y, y_predict) \
                                                           , recall_score(self.valid_y, y_predict)))
        
        if valid_f1 >= self.best_f1:
            self.best_f1 = valid_f1
            self.best_alpha = copy.deepcopy(self.alpha)
            self.best_b = self.b
            print('           get new model')
            
            if self.best_f1 >= 0.95:
                return True
            else:
                return False

In [4]:
def make_data_of_digit_i_and_j(x, y, digit_i, digit_j):
    pos_sample_num = 0
    neg_sample_num = 0
    _data = []
    
    for i in range(len(x)):
        if y[i] == digit_i and pos_sample_num < 1000:
            _data.append([x[i], 1])
            pos_sample_num += 1

        elif y[i] == digit_j and neg_sample_num < 1000:
            neg_sample_num += 1
            _data.append([x[i], -1])

        else:
            continue
        
    shuffle(_data)
    data =  []
    labels =[]
    
    for i in range(len(_data)):
        data.append(_data[i][0])
        labels.append(_data[i][1])
    
    return data, labels

In [9]:
def train_single_model(C, kernel_type):
    svm = SVM(C=C)
    
    train_data, train_labels = make_data_of_digit_i_and_j(X_train, y_train, 1, 2)
    valid_data, valid_labels = make_data_of_digit_i_and_j(X_valid, y_valid, 1, 2)
    test_data, test_labels = make_data_of_digit_i_and_j(X_test, y_test, 1, 2)
    
    train_data = np.array(train_data)
    valid_data = np.array(valid_data)
    test_data = np.array(test_data)
    
    svm.train(train_data, train_labels, valid_data, valid_labels, epochs=5, kernel_type=kernel_type)
        
    predict_list = []
    for i in tqdm(range(len(test_data))):
        p = svm.predict(test_data[i], opt='test')
        predict_list.append(p)
        
    print('test f1: {} precision: {} recall: {}'.format(f1_score(test_labels, predict_list) , \
                                                            precision_score(test_labels, predict_list) \
                                                           , recall_score(test_labels, predict_list)))
    
# single_model(0.1, 'rbf')
train_single_model(0.001, 'poly')
# single_model(10, 'rbf')
# single_model(100, 'rbf')
# single_model(1000, 'rbf')
# single_model(10000, 'rbf')
# single_model(100000, 'rbf')

  1%|▌                                                                              | 13/2000 [00:00<00:15, 126.55it/s]

start initing valid kernel matrix...


  0%|▎                                                                                | 9/2000 [00:00<00:25, 79.16it/s]

finish initing valid kernel matrix...
start initing kernel matrix...


  1%|▉                                                                              | 23/2000 [00:00<00:08, 223.90it/s]

...

finish initing E...
valid f1: 0.6812411847672778 precision: 0.5261437908496732 recall: 0.966
           get new model
valid f1: 0.6759040216289287 precision: 0.5104645227156712 recall: 1.0
valid f1: 0.8311148086522463 precision: 0.7115384615384616 recall: 0.999
           get new model
valid f1: 0.6798096532970768 precision: 0.5149330587023687 recall: 1.0


  0%|▍                                                                               | 10/2000 [00:00<00:21, 93.71it/s]

valid f1: 0.9715447154471545 precision: 0.987603305785124 recall: 0.956
           get new model


100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:19<00:00, 102.54it/s]


test f1: 0.9720670391061452 precision: 0.9876160990712074 recall: 0.957


In [ ]:
models = {}

for i in range(10):
    for j in range(i+1, 10):
        
        print('* train model of {} and {}'.format(i, j))
        
        svm = SVM(C=0.001)
    
        train_data, train_labels = make_data_of_digit_i_and_j(X_train, y_train, i, j)
        valid_data, valid_labels = make_data_of_digit_i_and_j(X_valid, y_valid, i, j)
        
        train_data = np.array(train_data)
        valid_data = np.array(valid_data)
        
        svm.train(train_data, train_labels, valid_data, valid_labels, epochs=5, kernel_type='poly')
        
        models[str(i) + str(j)] = svm

  1%|▌                                                                              | 14/2000 [00:00<00:14, 137.24it/s]

start initing valid kernel matrix...


  1%|▍                                                                              | 12/2000 [00:00<00:17, 115.70it/s]

finish initing valid kernel matrix...
start initing kernel matrix...


  2%|█▍                                                                             | 35/2000 [00:00<00:05, 345.72it/s]

...

finish initing valid kernel matrix...
start initing kernel matrix...


  3%|██▋                                                                            | 69/2000 [00:00<00:05, 348.57it/s]

finish initing kernel matrix...
start initing E...


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

finish initing E...
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1: 0.6666666666666666 precision: 0.5 recall: 1.0
           get new model
valid f1

In [ ]:
for model in models:
    print(model.best_f1)

In [ ]:
correct = 0
correct_list = []
error_list = []
for i in tqdm(range(len(X_test[:1000]))):
    prob_list = []
    for j in range(10):
        p = models[j].predict_of_probability(X_test[i])
        prob_list.append(p)
    
    p = np.argmax(prob_list)
    
    if p == y_test[i]:
        correct += 1
        correct_list.append(p)
    
    else:
        error_list.append([p, y_test[i]])
print('acc on test data of custom model: {}'.format(correct/len(X_test[:1000])))

In [ ]:
ans = []
for i in range(len(X_test[:1000])):
    ans.append(y_test[i])

In [ ]:
a = dict(Counter(correct_list))
b = dict(Counter(ans))
for i in range(10):
    if i in a.keys():
        print('recall of digit {}: {}'.format(i, a[i]/b[i]))
    else:
        print('recall of digit {}: {}'.format(i, 0.0))

In [ ]:
s = []
for error in error_list:
    key = str(error[1]) + '->' + str(error[0])
    s.append(key)

s.sort()
Counter(s)